In [32]:
import seg_metrics.seg_metrics as sg
import pandas as pd
import numpy as np
import nibabel as nib
import os
import pydicom
from skimage import morphology
import matplotlib.pyplot as plt
from skimage.measure import label
import cv2

In [33]:
def icc(Y, icc_type='ICC(2,1)'):
    ''' Calculate intraclass correlation coefficient

    ICC Formulas are based on:
    Shrout, P. E., & Fleiss, J. L. (1979). Intraclass correlations: uses in
    assessing rater reliability. Psychological bulletin, 86(2), 420.
    icc1:  x_ij = mu + beta_j + w_ij
    icc2/3:  x_ij = mu + alpha_i + beta_j + (ab)_ij + epsilon_ij
    Code modifed from nipype algorithms.icc
    https://github.com/nipy/nipype/blob/master/nipype/algorithms/icc.py

    Args:
        Y: The data Y are entered as a 'table' ie. subjects are in rows and repeated
            measures in columns
        icc_type: type of ICC to calculate. (ICC(2,1), ICC(2,k), ICC(3,1), ICC(3,k)) 
    Returns:
        ICC: (np.array) intraclass correlation coefficient
    '''

    [n, k] = Y.shape

    # Degrees of Freedom
    dfc = k - 1
    dfe = (n - 1) * (k-1)
    dfr = n - 1

    # Sum Square Total
    mean_Y = np.mean(Y)
    SST = ((Y - mean_Y) ** 2).sum()

    # create the design matrix for the different levels
    x = np.kron(np.eye(k), np.ones((n, 1)))  # sessions
    x0 = np.tile(np.eye(n), (k, 1))  # subjects
    X = np.hstack([x, x0])

    # Sum Square Error
    predicted_Y = np.dot(np.dot(np.dot(X, np.linalg.pinv(np.dot(X.T, X))),
                                X.T), Y.flatten('F'))
    residuals = Y.flatten('F') - predicted_Y
    SSE = (residuals ** 2).sum()

    MSE = SSE / dfe

    # Sum square column effect - between colums
    SSC = ((np.mean(Y, 0) - mean_Y) ** 2).sum() * n
    MSC = SSC / dfc  # / n (without n in SPSS results)

    # Sum Square subject effect - between rows/subjects
    SSR = SST - SSC - SSE
    MSR = SSR / dfr

    if icc_type == 'icc1':
        # ICC(2,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error +
        # k*(mean square columns - mean square error)/n)
        # ICC = (MSR - MSRW) / (MSR + (k-1) * MSRW)
        NotImplementedError("This method isn't implemented yet.")

    elif icc_type == 'ICC(2,1)' or icc_type == 'ICC(2,k)':
        # ICC(2,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error +
        # k*(mean square columns - mean square error)/n)
        if icc_type == 'ICC(2,k)':
            k = 1
        ICC = (MSR - MSE) / (MSR + (k-1) * MSE + k * (MSC - MSE) / n)

    elif icc_type == 'ICC(3,1)' or icc_type == 'ICC(3,k)':
        # ICC(3,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error)
        if icc_type == 'ICC(3,k)':
            k = 1
        ICC = (MSR - MSE) / (MSR + (k-1) * MSE)

    return ICC

In [34]:
def get_scores(metrics_df, dir_gt, dir_pred):
    volumes_gt = []
    volumes_pred = []
    for filename in metrics_df.filename:
        item = os.path.split(filename)[-1]
        filename_gt = os.path.join(dir_gt,item)
        filename_pred = os.path.join(dir_pred,item)
        nii_gt = nib.load(filename_gt)
        nii_pred = nib.load(filename_pred)
        array_gt = nii_gt.get_fdata()
        array_pred = nii_pred.get_fdata()
        volume_gt = np.sum(array_gt>0)
        volume_pred = np.sum(array_pred)
        volumes_gt.append(volume_gt)
        volumes_pred.append(volume_pred)
    metrics_df['volume_gt'] = volumes_gt
    metrics_df['volume_pred'] = volumes_pred

    icc_score = icc(np.column_stack((volumes_gt, volumes_pred)), icc_type='ICC(2,k)')
    
    scores_median = metrics_df[scorenames].median()
    scores_median['icc'] = icc_score
    
    Q1 = metrics_df[scorenames].quantile(0.25)
    Q3 = metrics_df[scorenames].quantile(0.75)
    scores_iqr = Q3 - Q1
    
    return scores_median, scores_iqr

In [35]:
def dirname2scores(dirname_gt, dirname_pred, filename_metrics):
    metrics = sg.write_metrics(labels=[1], gdth_path=dirname_gt, pred_path=dirname_pred, csv_file=filename_metrics)
    metrics_dataframe = pd.read_csv(filename_metrics)
    scores_median, scores_iqr = get_scores(metrics_dataframe, dirname_gt, dirname_pred)
    return scores_median, scores_iqr, metrics_dataframe

In [36]:
scorenames = ['dice', 'jaccard', 'precision', 'recall', 'fpr', 'fnr', 'vs', 'hd', 'msd', 'mdsd', 'stdsd', 'hd95']

## nnUNet

In [90]:
nifti_dirname_GT = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_GT"
nifti_dirname_GT_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_GT"

nifti_dirname_test = r"D:\Lisa\nnUNet_raw_data_base\nnUNet_results\Ts1"
nifti_dirname_t2w = r"D:\Lisa\nnUNet_raw_data_base\nnUNet_results\Ts2"
nifti_dirname_t1wce = r"D:\Lisa\nnUNet_raw_data_base\nnUNet_results\Ts3"
nifti_dirname_emc = r"D:\Lisa\nnUNet_raw_data_base\nnUNet_results\Ts4"

In [91]:
scores_median_test, scores_iqr_test = dirname2scores(nifti_dirname_GT, nifti_dirname_test, 'metrics_nnunet_test.csv')
scores_median_t2w, scores_iqr_t2w = dirname2scores(nifti_dirname_GT, nifti_dirname_t2w, 'metrics_nnunet_t2w.csv')
scores_median_t1wce, scores_iqr_t1wce = dirname2scores(nifti_dirname_GT, nifti_dirname_t1wce, 'metrics_nnunet_t1wce.csv')
scores_median_emc, scores_iqr_emc = dirname2scores(nifti_dirname_GT_emc, nifti_dirname_emc, 'metrics_nnunet_emc.csv')

Process UMCU036.nii.gz ...: : 24it [00:19,  1.23it/s]
Process UMCU036.nii.gz ...: : 24it [00:15,  1.53it/s]
Process UMCU036.nii.gz ...: : 24it [00:16,  1.48it/s]
Process EMC057.nii.gz ...: : 34it [00:24,  1.40it/s]


In [94]:
scores_df = pd.concat([scores_median_test, scores_iqr_test, 
                       scores_median_t2w, scores_iqr_t2w, 
                       scores_median_t1wce, scores_iqr_t1wce, 
                       scores_median_emc, scores_iqr_emc], axis=1)
scores_df.columns = ['test (median)', 'test (IQR)', 't2w (median)', 't2w (IQR)', 
                     't1wce (median)', 't1wce (IQR)', 'emc (median)', 'emc (IQR)']
scores_df

,test (median),test (IQR),t2w (median),t2w (IQR),t1wce (median),t1wce (IQR),emc (median),emc (IQR)
dice,0.915515,0.040113,0.892956,0.048952,0.852200,0.111599,0.872578,0.125272
jaccard,0.844221,0.067904,0.806619,0.079300,0.742651,0.166549,0.773968,0.187606
precision,0.926388,0.049276,0.923389,0.091353,0.877488,0.114831,0.979682,0.034998
recall,0.891519,0.096572,0.887612,0.118099,0.847257,0.189222,0.781861,0.162539
fpr,0.000416,0.000312,0.000464,0.000546,0.000748,0.000935,0.000179,0.000265
fnr,0.108481,0.096572,0.112388,0.118099,0.152743,0.189222,0.218138,0.162539
vs,0.069464,0.115188,0.013929,0.196363,0.013119,0.177423,0.213494,0.181101
hd,2.534831,5.266330,3.589805,6.920361,4.373492,9.153235,6.579737,4.545748
msd,0.192148,0.260591,0.310645,0.223993,0.434132,0.649802,0.537144,0.402031
mdsd,0.000000,0.000000,0.000000,0.000000,0.000000,0.075758,0.273449,0.386714


## 2D UNet

In [100]:
import shutil

In [101]:
sub_names_test = ['AMC012', 'AMC006', 'MUMC094', 'MUMC027', 'MUMC079', 'MUMC052', 'MUMC127', 'MUMC071', 'MUMC038',
                  'MUMC093', 'MUMC107', 'MUMC022', 'MUMC114', 'MUMC115', 'MUMC069', 'MUMC130', 'MUMC036', 'MUMC007', 
                  'MUMC059', 'MUMC080', 'UMCU036', 'UMCU025', 'UMCU008', 'UMCU034']

In [102]:
sub_names_emc = ['EMC003', 'EMC004', 'EMC005', 'EMC007', 'EMC008', 'EMC009', 'EMC011', 'EMC015', 'EMC018', 'EMC020', 
                 'EMC024', 'EMC027', 'EMC029', 'EMC031', 'EMC032', 'EMC034', 'EMC035', 'EMC036', 'EMC038', 'EMC041', 
                 'EMC042', 'EMC043', 'EMC045', 'EMC046', 'EMC047', 'EMC048', 'EMC049', 'EMC050', 'EMC051', 'EMC052', 
                 'EMC054', 'EMC055', 'EMC056', 'EMC057']

In [114]:
for sub_name in sub_names_test:
    
    sub_dir_name = os.path.join(r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti", sub_name)
    shutil.copy(sub_dir_name+'/mask_T2w_plaqueuqalp_clean_aug.nii.gz', 
                "C:/Users/E.Lavrova/Documents/GitHub/plaqueuqalp/res/nifti_compare/t2w_DCE_aug/"+sub_name+'.nii.gz')
    shutil.copy(sub_dir_name+'/mask_T1wCE_plaqueuqalp_clean_aug.nii.gz', 
                "C:/Users/E.Lavrova/Documents/GitHub/plaqueuqalp/res/nifti_compare/t1wce_DCE_aug/"+sub_name+'.nii.gz')

In [115]:
for sub_name in sub_names_emc:
    
    sub_dir_name = os.path.join(r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti", sub_name)
    shutil.copy(sub_dir_name+'/mask_T1w_plaqueuqalp_clean_aug.nii.gz', 
                "C:/Users/E.Lavrova/Documents/GitHub/plaqueuqalp/res/nifti_compare/emc_DCE_aug/"+sub_name+'.nii.gz')
    

In [116]:
nifti_dirname_GT = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_GT"
nifti_dirname_GT_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_GT"

nifti_dirname_test = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_DCE_aug"
nifti_dirname_t2w = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w_DCE_aug"
nifti_dirname_t1wce = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce_DCE_aug"
nifti_dirname_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_DCE_aug"

In [119]:
scores_median_test, scores_iqr_test, scores_df_test = dirname2scores(nifti_dirname_GT, nifti_dirname_test, 
                                                                     'metrics_2dunet_test.csv')
scores_median_t2w, scores_iqr_t2w, scores_df_t2w = dirname2scores(nifti_dirname_GT, nifti_dirname_t2w, 
                                                                  'metrics_2dunet_t2w.csv')
scores_median_t1wce, scores_iqr_t1wce, scores_df_t1wce = dirname2scores(nifti_dirname_GT, nifti_dirname_t1wce,
                                                                        'metrics_2dunet_t1wce.csv')
scores_median_emc, scores_iqr_emc, scores_df_emc = dirname2scores(nifti_dirname_GT_emc, nifti_dirname_emc, 
                                                                  'metrics_2dunet_emc.csv')

Process UMCU036.nii.gz ...: : 24it [00:17,  1.35it/s]
Process UMCU036.nii.gz ...: : 24it [00:17,  1.39it/s]
Process UMCU036.nii.gz ...: : 24it [00:17,  1.40it/s]
Process EMC057.nii.gz ...: : 34it [00:26,  1.30it/s]


In [120]:
scores_df = pd.concat([scores_median_test, scores_iqr_test, 
                       scores_median_t2w, scores_iqr_t2w, 
                       scores_median_t1wce, scores_iqr_t1wce, 
                       scores_median_emc, scores_iqr_emc], axis=1)
scores_df.columns = ['test (median)', 'test (IQR)', 't2w (median)', 't2w (IQR)', 
                     't1wce (median)', 't1wce (IQR)', 'emc (median)', 'emc (IQR)']
scores_df

,test (median),test (IQR),t2w (median),t2w (IQR),t1wce (median),t1wce (IQR),emc (median),emc (IQR)
dice,0.892105,0.129558,0.802391,0.254368,0.751613,0.217027,0.706643,0.397046
jaccard,0.805320,0.198874,0.670076,0.328289,0.602068,0.287709,0.546530,0.444579
precision,0.913918,0.065448,0.925862,0.106952,0.878188,0.116043,0.978192,0.042888
recall,0.871530,0.197892,0.720003,0.357763,0.682016,0.278775,0.556310,0.450156
fpr,0.000529,0.000519,0.000351,0.000488,0.000688,0.000701,0.000109,0.000116
fnr,0.128470,0.197892,0.279997,0.357763,0.317984,0.278775,0.443690,0.450156
vs,0.071442,0.233853,0.251306,0.401303,0.137979,0.370433,0.541320,0.662958
hd,6.538171,9.742688,5.497188,8.737045,9.334269,8.682934,9.088090,7.203315
msd,0.457897,0.662723,0.569441,0.980231,0.775220,1.136587,0.962968,1.068552
mdsd,0.000000,0.303029,0.151514,0.882391,0.303028,0.623943,0.546904,1.093801


## plaq-u-net

In [347]:
def mapslice2slice(map_array_slice, th=0.5):
    return morphology.remove_small_objects(map_array_slice>th, 50, connectivity=3).astype(np.uint8)

def filenames2array(filenames, emc=False):
    array_volume = np.zeros((im_height, im_height, len(filenames)), dtype=np.uint8)
    i = 0
    for filename in filenames:
        array_slice = np.load(filename)
        if emc:
            array_volume[..., i] = (cv2.resize(mapslice2slice(array_slice).copy(), 
                                              dsize=(im_height,im_height), 
                                              interpolation=cv2.INTER_CUBIC)>0).astype(np.uint8)
        else:
            array_volume[..., i] = mapslice2slice(array_slice)            
        i += 1
    return array_volume

def path2array(dir_name, sub, emc=False):
    
    sub_dir_name = os.path.join(dir_name, sub)
    filenames = []
    for item in os.listdir(sub_dir_name):
        filenames.append(os.path.join(sub_dir_name, item))
    filenames.sort()
    array = filenames2array(filenames, emc)
    
    return array


In [334]:
im_height = 512
im_width = 256

pixel_spacing = [0.303030, 0.303030]
pixel_spacing_emc = [0.546900/2, 0.546900/2]
slice_thickness = 2

def create_affine(pixel_spacing):
    affine = np.eye(4)
    affine[0, 0] = pixel_spacing[0]
    affine[1, 1] = pixel_spacing[1]
    affine[2, 2] = slice_thickness
    return affine

affine = create_affine(pixel_spacing)
affine_emc = create_affine(pixel_spacing_emc)

def array2halfarray(array_gt, array_pred):
    
    con_1 = array_gt[:, :array_gt.shape[1]//2].copy()
    con_2 = array_gt[:, array_gt.shape[1]//2:].copy()
    
    pred_1 = array_pred[:, :array_pred.shape[1]//2].copy()
    pred_2 = array_pred[:, array_pred.shape[1]//2:].copy()
    
    if (np.sum(con_1)>0)&(np.sum(con_2)>0):
        print (impth)
    elif np.sum(con_1)>0:
        con_output = con_1
        pred_output = pred_1
    elif np.sum(con_2)>0:
        con_output = np.fliplr(con_2)
        pred_output = np.fliplr(pred_2)
    return con_output, pred_output

In [335]:
dir_GT = r"C:\Users\E.Lavrova\Documents\GitHub\plaq-u-net\data"
dir_T1w = r"C:\Users\E.Lavrova\Documents\GitHub\plaq-u-net\res\maps\T1w\plaq-u-net_aug"
dir_T2w = r"C:\Users\E.Lavrova\Documents\GitHub\plaq-u-net\res\maps\T2w\plaq-u-net_aug"
dir_T1wCE = r"C:\Users\E.Lavrova\Documents\GitHub\plaq-u-net\res\maps\T1wCE\plaq-u-net_aug"

dir_nifti = r"C:\Users\E.Lavrova\Documents\GitHub\plaq-u-net\res\nifti"

In [336]:
def dcmpath2array(dcm_path, sub_name, emc=False):
    
    dcm_sub_dir = os.path.join(dcm_path, sub_name, 'MASSExport')
    
    dcm_filenames = []
    for item in os.listdir(dcm_sub_dir):
        dcm_filename = os.path.join(dcm_sub_dir, item)
        
        dcm_filenames.append(dcm_filename)
    dcm_filenames.sort()
    array_volume = np.zeros((im_height, im_height, len(dcm_filenames)), dtype=np.uint8)
    i = 0

    for dcm_filename in dcm_filenames:
        arr_dcm = pydicom.read_file(dcm_filename, force = True)
        arr_dcm.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian
        arr = arr_dcm.pixel_array
        if emc:
            arr_res = cv2.resize(arr.copy(), dsize=(im_height,im_height), interpolation=cv2.INTER_CUBIC)
            array_volume[..., i] = ((arr_res>0)&(arr_res<7)).astype(np.uint8)
        else:
            array_volume[..., i] = (arr[8:-8, 8:-8]>0).astype(np.uint8)
        i+=1
    return array_volume

In [414]:
def mask2labeledmask2d(mask_input):
    mask_labeled = label(mask_input)
    label_weights = []
    for l in range(1, np.max(mask_labeled)+1):
        mask_label = (mask_labeled==l).astype(np.uint8)
        weight_label = np.sum(mask_label*mask_input)
        rec_lw = {'label': l, 'weight': weight_label}
        label_weights.append(rec_lw)
    label_weights = pd.DataFrame(label_weights)
    n_labels = min(len(label_weights), 1)
    labels = []
    if len(label_weights)>0:
        label_weights.sort_values(by='weight', inplace=True, ascending = False)
        labels = np.array(label_weights['label'])[:n_labels]
    return np.isin(mask_labeled, labels).astype(np.uint8)

In [415]:
def mask2labeledmask(mask_input):
    mask_output = np.zeros(mask_input.shape, dtype=np.uint8)
    for i in range (0, mask_input.shape[2]):
        mask_output[..., i] = mask2labeledmask2d(mask_input[..., i])
    return mask_output

In [416]:
def rotate_array(array_input):
    dim = array_input.shape
    array_output = np.zeros((dim[1], dim[0], dim[2]))
    for i in range (0, dim[2]):
        array_output[..., i] = np.rot90(array_input[..., i], -1)
    return array_output

In [417]:
for sub_name in sub_names_test:
    
    array_GT = dcmpath2array(dir_GT, sub_name)
    array_T1w = path2array(dir_T1w, sub_name)
    array_T2w = path2array(dir_T2w, sub_name)
    array_T1wCE = path2array(dir_T1wCE, sub_name)
    
    array_GT_half, array_T1w_half = array2halfarray(array_GT, array_T1w)
    array_GT_half, array_T2w_half = array2halfarray(array_GT, array_T2w)
    array_GT_half, array_T1wCE_half = array2halfarray(array_GT, array_T1wCE)
    
    array_T1w_half_selected = mask2labeledmask(array_T1w_half)
    array_T2w_half_selected = mask2labeledmask(array_T2w_half)
    array_T1wCE_half_selected = mask2labeledmask(array_T1wCE_half)
    
    array_GT_half_rotated = rotate_array(array_GT_half)
    array_T1w_half_selected_rotated = rotate_array(array_T1w_half_selected)
    array_T2w_half_selected_rotated = rotate_array(array_T2w_half_selected)
    array_T1wCE_half_selected_rotated = rotate_array(array_T1wCE_half_selected)

    nifti_GT = nib.Nifti1Image(array_GT_half_rotated, affine=affine)
    nifti_T1w = nib.Nifti1Image(array_T1w_half_selected_rotated, affine=affine)
    nifti_T2w = nib.Nifti1Image(array_T2w_half_selected_rotated, affine=affine)
    nifti_T1wCE = nib.Nifti1Image(array_T1wCE_half_selected_rotated, affine=affine)

    nib.save(nifti_GT, os.path.join(dir_nifti, sub_name) + '/mask_GT.nii.gz')
    nib.save(nifti_T1w, os.path.join(dir_nifti, sub_name) + '/mask_T1w.nii.gz')
    nib.save(nifti_T2w, os.path.join(dir_nifti, sub_name) + '/mask_T2w.nii.gz')
    nib.save(nifti_T1wCE, os.path.join(dir_nifti, sub_name) + '/mask_T1wCE.nii.gz')
    
    

In [418]:
for sub_name in sub_names_emc:
    
    array_GT = dcmpath2array(dir_GT, sub_name, emc=True)
    array_T1w = path2array(dir_T1w, sub_name, emc=True)
    
    array_GT_half, array_T1w_half = array2halfarray(array_GT, array_T1w)
    
    array_T1w_half_selected = mask2labeledmask(array_T1w_half)
    
    array_GT_half_rotated = rotate_array(array_GT_half)
    array_T1w_half_selected_rotated = rotate_array(array_T1w_half_selected)

    nifti_GT = nib.Nifti1Image(array_GT_half_rotated, affine=affine)
    nifti_T1w = nib.Nifti1Image(array_T1w_half_selected_rotated, affine=affine)

    nib.save(nifti_GT, os.path.join(dir_nifti, sub_name) + '/mask_GT.nii.gz')
    nib.save(nifti_T1w, os.path.join(dir_nifti, sub_name) + '/mask_T1w.nii.gz')

In [419]:
for sub_name in sub_names_test:
    
    sub_dir_name = os.path.join(r"C:\Users\E.Lavrova\Documents\GitHub\plaq-u-net\res\nifti", sub_name)
    shutil.copy(sub_dir_name+'/mask_GT.nii.gz', 
                "C:/Users/E.Lavrova/Documents/GitHub/plaq-u-net/res/nifti_compare/test_gt/"+sub_name+'.nii.gz')
    shutil.copy(sub_dir_name+'/mask_T1w.nii.gz', 
                "C:/Users/E.Lavrova/Documents/GitHub/plaq-u-net/res/nifti_compare/test_plaqunet_aug/"+sub_name+'.nii.gz')
    shutil.copy(sub_dir_name+'/mask_T2w.nii.gz', 
                "C:/Users/E.Lavrova/Documents/GitHub/plaq-u-net/res/nifti_compare/t2w_plaqunet_aug/"+sub_name+'.nii.gz')
    shutil.copy(sub_dir_name+'/mask_T1wCE.nii.gz', 
                "C:/Users/E.Lavrova/Documents/GitHub/plaq-u-net/res/nifti_compare/t1wce_plaqunet_aug/"+sub_name+'.nii.gz')

In [420]:
for sub_name in sub_names_emc:
    
    sub_dir_name = os.path.join(r"C:\Users\E.Lavrova\Documents\GitHub\plaq-u-net\res\nifti", sub_name)
    shutil.copy(sub_dir_name+'/mask_GT.nii.gz', 
                "C:/Users/E.Lavrova/Documents/GitHub/plaq-u-net/res/nifti_compare/emc_gt/"+sub_name+'.nii.gz')
    shutil.copy(sub_dir_name+'/mask_T1w.nii.gz', 
                "C:/Users/E.Lavrova/Documents/GitHub/plaq-u-net/res/nifti_compare/emc_plaqunet_aug/"+sub_name+'.nii.gz')

In [421]:
nifti_dirname_GT = r"C:\Users\E.Lavrova\Documents\GitHub\plaq-u-net\res\nifti_compare\test_gt"
nifti_dirname_GT_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaq-u-net\res\nifti_compare\emc_gt"

nifti_dirname_test = r"C:\Users\E.Lavrova\Documents\GitHub\plaq-u-net\res\nifti_compare\test_plaqunet_aug"
nifti_dirname_t2w = r"C:\Users\E.Lavrova\Documents\GitHub\plaq-u-net\res\nifti_compare\t2w_plaqunet_aug"
nifti_dirname_t1wce = r"C:\Users\E.Lavrova\Documents\GitHub\plaq-u-net\res\nifti_compare\t1wce_plaqunet_aug"
nifti_dirname_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaq-u-net\res\nifti_compare\emc_plaqunet_aug"

In [422]:
scores_median_test, scores_iqr_test, scores_df_test = dirname2scores(nifti_dirname_GT, nifti_dirname_test, 
                                                                     'metrics_plaqunet_test.csv')
scores_median_t2w, scores_iqr_t2w, scores_df_t2w = dirname2scores(nifti_dirname_GT, nifti_dirname_t2w, 
                                                                  'metrics_plaqunet_t2w.csv')
scores_median_t1wce, scores_iqr_t1wce, scores_df_t1wce = dirname2scores(nifti_dirname_GT, nifti_dirname_t1wce,
                                                                        'metrics_plaqunet_t1wce.csv')
scores_median_emc, scores_iqr_emc, scores_df_emc = dirname2scores(nifti_dirname_GT_emc, nifti_dirname_emc, 
                                                                  'metrics_plaqunet_emc.csv')

Process UMCU036.nii.gz ...: : 24it [00:21,  1.11it/s]
Process UMCU036.nii.gz ...: : 24it [00:20,  1.15it/s]
Process UMCU036.nii.gz ...: : 24it [00:21,  1.13it/s]
Process EMC057.nii.gz ...: : 34it [00:30,  1.10it/s]


In [423]:
scores_df = pd.concat([scores_median_test, scores_iqr_test, 
                       scores_median_t2w, scores_iqr_t2w, 
                       scores_median_t1wce, scores_iqr_t1wce, 
                       scores_median_emc, scores_iqr_emc], axis=1)
scores_df.columns = ['test (median)', 'test (IQR)', 't2w (median)', 't2w (IQR)', 
                     't1wce (median)', 't1wce (IQR)', 'emc (median)', 'emc (IQR)']
scores_df

,test (median),test (IQR),t2w (median),t2w (IQR),t1wce (median),t1wce (IQR),emc (median),emc (IQR)
dice,0.602784,0.250277,0.363389,0.533983,0.533272,0.420358,0.447059,0.385483
jaccard,0.431421,0.237066,0.222067,0.437437,0.363905,0.373902,0.287880,0.330666
precision,0.603000,0.303601,0.393849,0.538000,0.603076,0.477683,0.546438,0.454640
recall,0.572169,0.230407,0.370048,0.502364,0.498712,0.390182,0.407134,0.351261
fpr,0.002412,0.002061,0.003500,0.004917,0.002152,0.003096,0.002365,0.003090
fnr,0.427831,0.230407,0.629952,0.502364,0.501288,0.390182,0.592866,0.351261
vs,0.032835,0.189686,0.009799,0.245772,0.103217,0.193699,0.316732,0.365412
hd,47.113661,20.160178,53.157366,24.353509,29.665457,29.272262,43.230659,27.104155
msd,7.776412,9.880072,12.985354,16.899622,5.818379,5.415099,7.987709,9.791983
mdsd,1.000837,3.410544,3.049180,20.355970,1.053263,5.089193,2.503984,4.786559


## nnUnet+plaq-u-net_0

In [19]:
nifti_dirname_GT = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_GT"
nifti_dirname_GT_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_GT"

nifti_dirname_test = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_plaqunet_0"
nifti_dirname_t2w = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w_plaqunet_0"
nifti_dirname_t1wce = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce_plaqunet_0"
nifti_dirname_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_plaqunet_0"

In [20]:
scores_median_test, scores_iqr_test, scores_df_test = dirname2scores(nifti_dirname_GT, nifti_dirname_test, 
                                                                     'metrics_nnunet_test.csv')
scores_median_t2w, scores_iqr_t2w, scores_df_t2w = dirname2scores(nifti_dirname_GT, nifti_dirname_t2w, 
                                                                  'metrics_nnunet_t2w.csv')
scores_median_t1wce, scores_iqr_t1wce, scores_df_t1wce = dirname2scores(nifti_dirname_GT, nifti_dirname_t1wce, 
                                                                        'metrics_nnunet_t1wce.csv')
scores_median_emc, scores_iqr_emc, scores_df_emc = dirname2scores(nifti_dirname_GT_emc, nifti_dirname_emc, 
                                                                  'metrics_nnunet_emc.csv')

Process UMCU036.nii.gz ...: : 24it [00:16,  1.49it/s]
Process UMCU036.nii.gz ...: : 24it [00:16,  1.47it/s]
Process UMCU036.nii.gz ...: : 24it [00:16,  1.47it/s]
Process EMC057.nii.gz ...: : 34it [00:23,  1.44it/s]


In [21]:
scores_df = pd.concat([scores_median_test, scores_iqr_test, 
                       scores_median_t2w, scores_iqr_t2w, 
                       scores_median_t1wce, scores_iqr_t1wce, 
                       scores_median_emc, scores_iqr_emc
                      ], axis=1)
scores_df.columns = ['test (median)', 'test (IQR)',
                     't2w (median)', 't2w (IQR)', 
                     't1wce (median)', 't1wce (IQR)', 'emc (median)', 'emc (IQR)'
                    ]
scores_df

,test (median),test (IQR),t2w (median),t2w (IQR),t1wce (median),t1wce (IQR),emc (median),emc (IQR)
dice,0.918981,0.049565,0.883187,0.053858,0.846723,0.141487,0.867752,0.091954
jaccard,0.850111,0.082718,0.790853,0.087398,0.734239,0.206714,0.766398,0.140128
precision,0.934899,0.053017,0.902210,0.114515,0.879899,0.159107,0.971845,0.061717
recall,0.881139,0.097286,0.879967,0.131299,0.827035,0.216220,0.779130,0.148214
fpr,0.000389,0.000265,0.000590,0.000663,0.000704,0.001100,0.000232,0.000455
fnr,0.118861,0.097286,0.120033,0.131299,0.172965,0.216220,0.220870,0.148214
vs,0.074665,0.114309,0.011216,0.202993,0.007452,0.205934,0.191117,0.157887
hd,2.278981,1.341168,3.086866,4.923934,4.206963,5.731180,7.088575,4.892117
msd,0.187475,0.219930,0.326324,0.415110,0.499309,0.693994,0.508131,0.309470
mdsd,0.000000,0.000000,0.000000,0.000000,0.000000,0.334405,0.273449,0.273453


## nnUnet+plaq-u-net_1

In [22]:
nifti_dirname_GT = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_GT"
nifti_dirname_GT_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_GT"

nifti_dirname_test = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_plaqunet_1"
nifti_dirname_t2w = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w_plaqunet_1"
nifti_dirname_t1wce = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce_plaqunet_1"
nifti_dirname_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_plaqunet_1"

In [23]:
scores_median_test, scores_iqr_test, scores_df_test = dirname2scores(nifti_dirname_GT, nifti_dirname_test, 
                                                                     'metrics_nnunet_test.csv')
scores_median_t2w, scores_iqr_t2w, scores_df_t2w = dirname2scores(nifti_dirname_GT, nifti_dirname_t2w, 
                                                                  'metrics_nnunet_t2w.csv')
scores_median_t1wce, scores_iqr_t1wce, scores_df_t1wce = dirname2scores(nifti_dirname_GT, nifti_dirname_t1wce, 
                                                                        'metrics_nnunet_t1wce.csv')
scores_median_emc, scores_iqr_emc, scores_df_emc = dirname2scores(nifti_dirname_GT_emc, nifti_dirname_emc, 
                                                                  'metrics_nnunet_emc.csv')

Process UMCU036.nii.gz ...: : 24it [00:15,  1.51it/s]
Process UMCU036.nii.gz ...: : 24it [00:16,  1.49it/s]
Process UMCU036.nii.gz ...: : 24it [00:16,  1.48it/s]
Process EMC057.nii.gz ...: : 34it [00:23,  1.45it/s]


In [24]:
scores_df = pd.concat([scores_median_test, scores_iqr_test, 
                       scores_median_t2w, scores_iqr_t2w, 
                       scores_median_t1wce, scores_iqr_t1wce, 
                       scores_median_emc, scores_iqr_emc
                      ], axis=1)
scores_df.columns = ['test (median)', 'test (IQR)',
                     't2w (median)', 't2w (IQR)', 
                     't1wce (median)', 't1wce (IQR)', 'emc (median)', 'emc (IQR)'
                    ]
scores_df

,test (median),test (IQR),t2w (median),t2w (IQR),t1wce (median),t1wce (IQR),emc (median),emc (IQR)
dice,0.917059,0.046400,0.889846,0.043432,0.854064,0.120159,0.864657,0.094141
jaccard,0.846835,0.078366,0.801553,0.070952,0.745495,0.179846,0.761609,0.143511
precision,0.935424,0.049023,0.928637,0.080770,0.885426,0.111299,0.981703,0.017900
recall,0.886693,0.098787,0.871683,0.105668,0.855175,0.197407,0.767949,0.149095
fpr,0.000403,0.000299,0.000414,0.000469,0.000712,0.000874,0.000162,0.000217
fnr,0.113307,0.098787,0.128317,0.105668,0.144825,0.197407,0.232051,0.149095
vs,0.078616,0.138643,0.027879,0.174818,0.001197,0.186080,0.230658,0.168252
hd,2.333002,1.446100,3.160275,2.997844,4.358794,5.416116,8.020160,5.991802
msd,0.176275,0.215433,0.318330,0.125242,0.425362,0.614794,0.571539,0.354170
mdsd,0.000000,0.000000,0.000000,0.000000,0.000000,0.091447,0.273449,0.358400


## nnUnet+plaq-u-net_2

In [29]:
nifti_dirname_GT = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_GT"
nifti_dirname_GT_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_GT"

nifti_dirname_test = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_plaqunet_2"
nifti_dirname_t2w = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w_plaqunet_2"
nifti_dirname_t1wce = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce_plaqunet_2"
nifti_dirname_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_plaqunet_2"

In [30]:
scores_median_test, scores_iqr_test, scores_df_test = dirname2scores(nifti_dirname_GT, nifti_dirname_test, 
                                                                     'metrics_nnunet_test.csv')
scores_median_t2w, scores_iqr_t2w, scores_df_t2w = dirname2scores(nifti_dirname_GT, nifti_dirname_t2w, 
                                                                  'metrics_nnunet_t2w.csv')
scores_median_t1wce, scores_iqr_t1wce, scores_df_t1wce = dirname2scores(nifti_dirname_GT, nifti_dirname_t1wce, 
                                                                        'metrics_nnunet_t1wce.csv')
scores_median_emc, scores_iqr_emc, scores_df_emc = dirname2scores(nifti_dirname_GT_emc, nifti_dirname_emc, 
                                                                  'metrics_nnunet_emc.csv')

Process UMCU036.nii.gz ...: : 24it [00:15,  1.51it/s]
Process UMCU036.nii.gz ...: : 24it [00:15,  1.52it/s]
Process UMCU036.nii.gz ...: : 24it [00:15,  1.53it/s]
Process EMC057.nii.gz ...: : 34it [00:22,  1.49it/s]


In [31]:
scores_df = pd.concat([scores_median_test, scores_iqr_test, 
                       scores_median_t2w, scores_iqr_t2w, 
                       scores_median_t1wce, scores_iqr_t1wce, 
                       scores_median_emc, scores_iqr_emc
                      ], axis=1)
scores_df.columns = ['test (median)', 'test (IQR)',
                     't2w (median)', 't2w (IQR)', 
                     't1wce (median)', 't1wce (IQR)', 'emc (median)', 'emc (IQR)'
                    ]
scores_df

,test (median),test (IQR),t2w (median),t2w (IQR),t1wce (median),t1wce (IQR),emc (median),emc (IQR)
dice,0.912684,0.044626,0.897129,0.048594,0.846618,0.145073,0.875964,0.084314
jaccard,0.839402,0.075247,0.813449,0.078749,0.734051,0.213411,0.779309,0.129939
precision,0.922880,0.063428,0.903296,0.095638,0.856443,0.164844,0.975290,0.072444
recall,0.893836,0.080846,0.904232,0.113706,0.836544,0.218494,0.801094,0.132171
fpr,0.000519,0.000337,0.000563,0.000615,0.000788,0.001256,0.000193,0.000621
fnr,0.106164,0.080846,0.095767,0.113706,0.163456,0.218494,0.198906,0.132171
vs,0.035946,0.112681,-0.004099,0.180548,-0.030542,0.175256,0.190421,0.157661
hd,2.606019,4.166997,3.568127,7.670254,4.644310,9.098051,7.428508,4.947242
msd,0.207168,0.266284,0.323962,0.160132,0.568087,0.735733,0.531661,0.435955
mdsd,0.000000,0.000000,0.000000,0.000000,0.000000,0.303026,0.273449,0.273453


## nnUnet+plaq-u-net+ul_0

In [38]:
nifti_dirname_GT = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_GT"
nifti_dirname_GT_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_GT"

nifti_dirname_test = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_plaqumap_0"
nifti_dirname_t2w = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w_plaqumap_0"
nifti_dirname_t1wce = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce_plaqumap_0"
nifti_dirname_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_plaqumap_0"

In [39]:
scores_median_test, scores_iqr_test, scores_df_test = dirname2scores(nifti_dirname_GT, nifti_dirname_test, 
                                                                     'metrics_test_plaqumap_0.csv')
scores_median_t2w, scores_iqr_t2w, scores_df_t2w = dirname2scores(nifti_dirname_GT, nifti_dirname_t2w, 
                                                                  'metrics_t2w_plaqumap_0.csv')
scores_median_t1wce, scores_iqr_t1wce, scores_df_t1wce = dirname2scores(nifti_dirname_GT, nifti_dirname_t1wce, 
                                                                        'metrics_t1wce_plaqumap_0.csv')
scores_median_emc, scores_iqr_emc, scores_df_emc = dirname2scores(nifti_dirname_GT_emc, nifti_dirname_emc, 
                                                                  'metrics_emc_plaqumap_0.csv')

Process UMCU036.nii.gz ...: : 24it [00:15,  1.55it/s]
Process UMCU036.nii.gz ...: : 24it [00:15,  1.56it/s]
Process UMCU036.nii.gz ...: : 24it [00:15,  1.57it/s]
Process EMC057.nii.gz ...: : 34it [00:22,  1.52it/s]


In [40]:
scores_df = pd.concat([scores_median_test, scores_iqr_test, 
                       scores_median_t2w, scores_iqr_t2w, 
                       scores_median_t1wce, scores_iqr_t1wce, 
                       scores_median_emc, scores_iqr_emc
                      ], axis=1)
scores_df.columns = ['test (median)', 'test (IQR)',
                     't2w (median)', 't2w (IQR)', 
                     't1wce (median)', 't1wce (IQR)', 'emc (median)', 'emc (IQR)'
                    ]
scores_df

,test (median),test (IQR),t2w (median),t2w (IQR),t1wce (median),t1wce (IQR),emc (median),emc (IQR)
dice,0.898164,0.048554,0.859112,0.070415,0.824533,0.157944,0.901359,0.050493
jaccard,0.815157,0.079800,0.753026,0.109025,0.701643,0.218846,0.820431,0.083170
precision,0.830368,0.098621,0.788056,0.143906,0.752530,0.219859,0.894426,0.120439
recall,0.964531,0.043219,0.941212,0.085936,0.907480,0.193797,0.907695,0.094347
fpr,0.001096,0.000587,0.001618,0.001116,0.001804,0.002029,0.001032,0.001432
fnr,0.035469,0.043219,0.058788,0.085936,0.092520,0.193797,0.092305,0.094346
vs,-0.147155,0.124052,-0.180343,0.204207,-0.217297,0.208185,0.000351,0.143146
hd,2.476111,2.445508,3.970852,7.028518,4.201960,6.611514,6.557976,4.414873
msd,0.242877,0.224283,0.497482,0.392880,0.662738,0.617449,0.441087,0.308736
mdsd,0.000000,0.075756,0.303028,0.303032,0.303031,0.606060,0.000000,0.273449


## nnUnet+plaq-u-net+ul_1

In [41]:
nifti_dirname_GT = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_GT"
nifti_dirname_GT_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_GT"

nifti_dirname_test = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_plaqumap_1"
nifti_dirname_t2w = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w_plaqumap_1"
nifti_dirname_t1wce = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce_plaqumap_1"
nifti_dirname_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_plaqumap_1"

In [42]:
scores_median_test, scores_iqr_test, scores_df_test = dirname2scores(nifti_dirname_GT, nifti_dirname_test, 
                                                                     'metrics_test_plaqumap_1.csv')
scores_median_t2w, scores_iqr_t2w, scores_df_t2w = dirname2scores(nifti_dirname_GT, nifti_dirname_t2w, 
                                                                  'metrics_t2w_plaqumap_1.csv')
scores_median_t1wce, scores_iqr_t1wce, scores_df_t1wce = dirname2scores(nifti_dirname_GT, nifti_dirname_t1wce, 
                                                                        'metrics_t1wce_plaqumap_1.csv')
scores_median_emc, scores_iqr_emc, scores_df_emc = dirname2scores(nifti_dirname_GT_emc, nifti_dirname_emc, 
                                                                  'metrics_emc_plaqumap_1.csv')

Process UMCU036.nii.gz ...: : 24it [00:15,  1.55it/s]
Process UMCU036.nii.gz ...: : 24it [00:15,  1.59it/s]
Process UMCU036.nii.gz ...: : 24it [00:15,  1.56it/s]
Process EMC057.nii.gz ...: : 34it [00:21,  1.55it/s]


In [43]:
scores_df = pd.concat([scores_median_test, scores_iqr_test, 
                       scores_median_t2w, scores_iqr_t2w, 
                       scores_median_t1wce, scores_iqr_t1wce, 
                       scores_median_emc, scores_iqr_emc
                      ], axis=1)
scores_df.columns = ['test (median)', 'test (IQR)',
                     't2w (median)', 't2w (IQR)', 
                     't1wce (median)', 't1wce (IQR)', 'emc (median)', 'emc (IQR)'
                    ]
scores_df

,test (median),test (IQR),t2w (median),t2w (IQR),t1wce (median),t1wce (IQR),emc (median),emc (IQR)
dice,0.900084,0.047796,0.874854,0.079263,0.835785,0.143815,0.910844,0.054848
jaccard,0.818320,0.079015,0.777570,0.122331,0.717921,0.201724,0.836284,0.090267
precision,0.843362,0.082542,0.812839,0.160227,0.777255,0.166752,0.904297,0.085474
recall,0.962692,0.043920,0.931460,0.089308,0.898659,0.202763,0.912031,0.092659
fpr,0.001063,0.000464,0.001402,0.001095,0.001716,0.001759,0.000934,0.000918
fnr,0.037308,0.043920,0.068540,0.089308,0.101341,0.202763,0.087969,0.092659
vs,-0.116067,0.107506,-0.157901,0.203495,-0.202882,0.146518,0.005607,0.147587
hd,2.447043,2.431890,4.038561,6.328425,4.155916,5.759914,6.528800,4.369030
msd,0.254958,0.167582,0.469096,0.489632,0.588177,0.798920,0.433332,0.312293
mdsd,0.000000,0.075756,0.151514,0.303032,0.303031,0.472928,0.000000,0.273448


## nnUnet+plaq-u-net+ul_2

In [44]:
nifti_dirname_GT = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_GT"
nifti_dirname_GT_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_GT"

nifti_dirname_test = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_plaqumap_2"
nifti_dirname_t2w = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w_plaqumap_2"
nifti_dirname_t1wce = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce_plaqumap_2"
nifti_dirname_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_plaqumap_2"

In [45]:
scores_median_test, scores_iqr_test, scores_df_test = dirname2scores(nifti_dirname_GT, nifti_dirname_test, 
                                                                     'metrics_test_plaqumap_2.csv')
scores_median_t2w, scores_iqr_t2w, scores_df_t2w = dirname2scores(nifti_dirname_GT, nifti_dirname_t2w, 
                                                                  'metrics_t2w_plaqumap_2.csv')
scores_median_t1wce, scores_iqr_t1wce, scores_df_t1wce = dirname2scores(nifti_dirname_GT, nifti_dirname_t1wce, 
                                                                        'metrics_t1wce_plaqumap_2.csv')
scores_median_emc, scores_iqr_emc, scores_df_emc = dirname2scores(nifti_dirname_GT_emc, nifti_dirname_emc, 
                                                                  'metrics_emc_plaqumap_2.csv')

Process UMCU036.nii.gz ...: : 24it [00:15,  1.59it/s]
Process UMCU036.nii.gz ...: : 24it [00:15,  1.58it/s]
Process UMCU036.nii.gz ...: : 24it [00:15,  1.57it/s]
Process EMC057.nii.gz ...: : 34it [00:22,  1.52it/s]


In [46]:
scores_df = pd.concat([scores_median_test, scores_iqr_test, 
                       scores_median_t2w, scores_iqr_t2w, 
                       scores_median_t1wce, scores_iqr_t1wce, 
                       scores_median_emc, scores_iqr_emc
                      ], axis=1)
scores_df.columns = ['test (median)', 'test (IQR)',
                     't2w (median)', 't2w (IQR)', 
                     't1wce (median)', 't1wce (IQR)', 'emc (median)', 'emc (IQR)'
                    ]
scores_df

,test (median),test (IQR),t2w (median),t2w (IQR),t1wce (median),t1wce (IQR),emc (median),emc (IQR)
dice,0.899202,0.047192,0.868910,0.075668,0.827824,0.147975,0.902461,0.055593
jaccard,0.816865,0.078045,0.768207,0.117625,0.706711,0.206599,0.822259,0.091905
precision,0.827153,0.091742,0.809267,0.148160,0.756033,0.178013,0.896634,0.097351
recall,0.964426,0.046673,0.938660,0.090008,0.906385,0.197671,0.912260,0.097956
fpr,0.001113,0.000636,0.001494,0.001014,0.001774,0.002019,0.001004,0.001302
fnr,0.035573,0.046673,0.061340,0.090008,0.093615,0.197671,0.087740,0.097956
vs,-0.146663,0.120697,-0.145727,0.195681,-0.206635,0.221600,-0.011306,0.150887
hd,2.383226,2.546489,3.589805,7.293672,4.155916,7.013265,6.545484,4.406245
msd,0.244557,0.224731,0.462735,0.469927,0.666680,0.629333,0.439888,0.316635
mdsd,0.000000,0.000000,0.303026,0.303032,0.303029,0.606058,0.000000,0.273449


## nnUnet+plaq-u-net+ul_3

In [47]:
nifti_dirname_GT = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_GT"
nifti_dirname_GT_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_GT"

nifti_dirname_test = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test_plaqumap_3"
nifti_dirname_t2w = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w_plaqumap_3"
nifti_dirname_t1wce = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce_plaqumap_3"
nifti_dirname_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc_plaqumap_3"

In [48]:
scores_median_test, scores_iqr_test, scores_df_test = dirname2scores(nifti_dirname_GT, nifti_dirname_test, 
                                                                     'metrics_test_plaqumap_3.csv')
scores_median_t2w, scores_iqr_t2w, scores_df_t2w = dirname2scores(nifti_dirname_GT, nifti_dirname_t2w, 
                                                                  'metrics_t2w_plaqumap_3.csv')
scores_median_t1wce, scores_iqr_t1wce, scores_df_t1wce = dirname2scores(nifti_dirname_GT, nifti_dirname_t1wce, 
                                                                        'metrics_t1wce_plaqumap_3.csv')
scores_median_emc, scores_iqr_emc, scores_df_emc = dirname2scores(nifti_dirname_GT_emc, nifti_dirname_emc, 
                                                                  'metrics_emc_plaqumap_3.csv')

Process UMCU036.nii.gz ...: : 24it [00:14,  1.61it/s]
Process UMCU036.nii.gz ...: : 24it [00:15,  1.58it/s]
Process UMCU036.nii.gz ...: : 24it [00:15,  1.59it/s]
Process EMC057.nii.gz ...: : 34it [00:22,  1.54it/s]


In [49]:
scores_df = pd.concat([scores_median_test, scores_iqr_test, 
                       scores_median_t2w, scores_iqr_t2w, 
                       scores_median_t1wce, scores_iqr_t1wce, 
                       scores_median_emc, scores_iqr_emc
                      ], axis=1)
scores_df.columns = ['test (median)', 'test (IQR)',
                     't2w (median)', 't2w (IQR)', 
                     't1wce (median)', 't1wce (IQR)', 'emc (median)', 'emc (IQR)'
                    ]
scores_df

,test (median),test (IQR),t2w (median),t2w (IQR),t1wce (median),t1wce (IQR),emc (median),emc (IQR)
dice,0.900209,0.052524,0.850773,0.082538,0.817459,0.155664,0.902055,0.073890
jaccard,0.818530,0.086355,0.740501,0.127061,0.691331,0.215393,0.821592,0.119525
precision,0.826656,0.111134,0.772450,0.148309,0.755188,0.230983,0.881841,0.130491
recall,0.963358,0.035713,0.941705,0.089945,0.904854,0.181308,0.912199,0.086190
fpr,0.001069,0.001057,0.001617,0.001460,0.001771,0.002095,0.001062,0.001835
fnr,0.036642,0.035713,0.058294,0.089945,0.095146,0.181308,0.087801,0.086190
vs,-0.162380,0.149183,-0.200128,0.199356,-0.224872,0.268147,-0.024232,0.140851
hd,2.280067,3.545588,3.610012,7.427546,4.308036,6.492209,6.471973,4.391632
msd,0.238862,0.228776,0.483417,0.538445,0.659390,0.637715,0.470040,0.305298
mdsd,0.000000,0.000000,0.303026,0.303032,0.303031,0.606064,0.000000,0.273449
